In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [4]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only, encoder_with_convs_and_symmetry_new
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tflearn.layers.conv import avg_pool_1d
from pc_completions.evaluation import basic_comletion_measures

from tflearn.layers.conv import conv_1d
from tflearn.layers.normalization import batch_normalization
from testing_encoder_decoder import decoder_with_convs_only

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
n_pc_points = 2048
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))

In [7]:
do_training = True
load_pretrained_model = False
load_epoch = None
random_seed = 42
loss = 'chamfer'
training_epochs = 250
batch_size = 50
learning_rate = 0.0005

In [8]:
train_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt')
val_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt')
test_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt')

splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)
tr_files = splitter.load_splits(train_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(tr_files, n_threads=20, loader=snc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
val_files = splitter.load_splits(val_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(val_files, n_threads=20, loader=snc_loader, verbose=True)
val_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
test_files = splitter.load_splits(test_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(test_files, n_threads=20, loader=snc_loader, verbose=True)
test_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.


In [21]:
b_norm = True
encoder_args = {'n_filters': [64, 128, 128, 256, 512],
                'filter_sizes': [9, 3, 2, 2, 1],
                'strides': [1, 1, 1, 1, 1],
                'b_norm': b_norm,
                'pool': avg_pool_1d,
                'pool_sizes': [4, None, 2, None, 2],
                }


decoder_args = {'layer_sizes': [32, 128, 512, 2048*3],
                'b_norm': b_norm,
                'non_linearity': None,
                }

# # layer = tf.placeholder(tf.float32, [None, n_pc_points, 3])
# # layer = encoder_with_convs_and_symmetry_new(layer, **encoder_args)
# # convolutional_decoder(layer)

# def encoder_with_convs_and_symmetry_and_fc(in_signal, fc_nout, args_of_patrial={}):
#     layer = encoder_with_convs_and_symmetry(in_signal, **args_of_patrial)
#     layer = fully_connected(layer, fc_nout, activation='relu', weights_init='xavier')
#     return layer


In [ ]:
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings/temp')

conf = Conf(
            n_input = [n_pc_points, 3],
            loss = loss,
            training_epochs = training_epochs,
            batch_size = batch_size,
            denoising = False,
            learning_rate = learning_rate,
            train_dir = train_dir,             
            loss_display_step = 1,
            saver_step = None,
            z_rotate = False,
            encoder = encoder_with_convs_and_symmetry_new, 
            decoder = decoder_with_fc_only,
            encoder_args = encoder_args,
            decoder_args = decoder_args
           )

conf.experiment_name = 'Test'
conf.held_out_step = 5
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

print ae.trainable_parameters()

if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout, held_out_data=val_data)
    fout.close()

conv params =  1792
bnorm params =  128
Tensor("Test_1/AvgPool1D/Squeeze:0", shape=(?, 512, 64), dtype=float32) 32768
conv params =  24704
bnorm params =  256
Tensor("Test_1/Relu_1:0", shape=(?, 512, 128), dtype=float32) 65536
conv params =  32896
bnorm params =  256
Tensor("Test_1/AvgPool1D_1/Squeeze:0", shape=(?, 256, 128), dtype=float32) 32768
conv params =  65792
bnorm params =  512
Tensor("Test_1/Relu_3:0", shape=(?, 256, 256), dtype=float32) 65536
conv params =  131584
bnorm params =  1024
Tensor("Test_1/AvgPool1D_2/Squeeze:0", shape=(?, 128, 512), dtype=float32) 65536
Tensor("Test_1/Max:0", shape=(?, 512), dtype=float32) 512
3498848
('Epoch:', '0001', 'training time (minutes)=', '0.0739', 'loss=', '0.011796080')
('Epoch:', '0002', 'training time (minutes)=', '0.0633', 'loss=', '0.005358907')
('Epoch:', '0003', 'training time (minutes)=', '0.0651', 'loss=', '0.004707753')
('Epoch:', '0004', 'training time (minutes)=', '0.0656', 'loss=', '0.004102468')
('Epoch:', '0005', 'training